To perform limb detection, we will be using a modified MPII Human Pose dataset. First, we'll need to download the entire dataset. This file seeks to filter and shrink this dataset into a smaller subset for easier use. This does not update the dataset to have bounding boxes either.

In [1]:
!wget https://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1.tar.gz
!wget https://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1_u12_2.zip

--2021-04-02 20:36:28--  https://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1.tar.gz
Resolving datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)... 139.19.206.177
Connecting to datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)|139.19.206.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12088943206 (11G) [application/x-gzip]
Saving to: ‘mpii_human_pose_v1.tar.gz’

mpii_human_pose_v1. 100%[===================>]  11.26G  10.5MB/s    in 18m 14s 

2021-04-02 20:54:44 (10.5 MB/s) - ‘mpii_human_pose_v1.tar.gz’ saved [12088943206/12088943206]

--2021-04-02 20:54:44--  https://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1_u12_2.zip
Resolving datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)... 139.19.206.177
Connecting to datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)|139.19.206.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12340483 (12M) [application/zip]
Saving to: ‘mpii_hu

In [2]:
!tar xzf mpii_human_pose_v1.tar.gz


/bin/bash: unxip: command not found


In [4]:
!unzip mpii_human_pose_v1_u12_2.zip

Archive:  mpii_human_pose_v1_u12_2.zip
   creating: mpii_human_pose_v1_u12_2/
  inflating: mpii_human_pose_v1_u12_2/bsd.txt  
  inflating: mpii_human_pose_v1_u12_2/mpii_human_pose_v1_u12_1.mat  
  inflating: mpii_human_pose_v1_u12_2/README.md  


In [140]:
import pandas as pd, csv
import os
import shutil
import numpy as np
import scipy.io as sio
from tqdm.auto import tqdm

The MPII dataset is stored in a Matlab .mat file structure, and so must be interpreted.

In [111]:
mat = sio.loadmat('mpii_human_pose_v1_u12_2/mpii_human_pose_v1_u12_1.mat', struct_as_record=False)
joints = ['r ankle_X','r ankle_Y','r knee_X','r knee_Y','r hip_X','r hip_Y','l hip_X','l hip_Y','l knee_X','l knee_Y','l ankle_X','l ankle_Y','pelvis_X','pelvis_Y','thorax_X','thorax_Y','upper neck_X','upper neck_Y','head top_X',
           'head top_Y','r wrist_X','r wrist_Y','r elbow_X','r elbow_Y','r shoulder_X','r shoulder_Y','l shoulder_X','l shoulder_Y','l elbow_X','l elbow_Y','l wrist_X','l wrist_Y']

In [112]:
data = mat['RELEASE']
print(data, type(data), data.shape)
data = data[0,0]


[[<scipy.io.matlab.mio5_params.mat_struct object at 0x7f4294d7bb50>]] <class 'numpy.ndarray'> (1, 1)


In [18]:
data._fieldnames

['annolist', 'img_train', 'version', 'single_person', 'act', 'video_list']

In [113]:
annolist = data.__dict__['annolist']
annolist

array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x7f4294d7b810>,
        ...,
      dtype=object)

In [114]:
img_train = data.__dict__['img_train']

In [115]:
categories = data.__dict__['act']

##Experimenting with scipy's .mat interpretation

In [100]:
cat = data.__dict__['act']
cat

array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x7f4297c520d0>],
       ...,
      dtype=object)

In [104]:
cat[4][0].__dict__['cat_name']

array(['sports'], dtype='<U6')

In [66]:
annolist[0,0]

In [67]:
annolist[0,5]

In [68]:
annorect = annolist[0,5].__dict__['annorect']
annorect

array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x7f42c4b4a690>,
      dtype=object)

In [69]:
annorect.shape[1]

3

In [70]:
annorect[0,1]

In [71]:
annorect[0,0].__dict__['x1'][0][0]

607

In [72]:
points = annorect[0,0].__dict__['annopoints']
points = points[0][0].__dict__['point']
id = points[0,0].__dict__['id']
id

array([[2]], dtype=uint8)

In [73]:
points.shape[1]

12

In [54]:
for point in points[0]:
  print(point.__dict__['id'][0,0])


2
3
6
7
8
9
10
11
12
13
14
15


##Processing the .mat file

In [149]:
datapoints = {
    "NAME": list(),
    'r ankle_X': list(),
    'r ankle_Y': list(),
    'r knee_X': list(),
    'r knee_Y': list(),
    'r hip_X': list(),
    'r hip_Y': list(),
    'l hip_X': list(),
    'l hip_Y': list(),
    'l knee_X': list(),
    'l knee_Y': list(),
    'l ankle_X': list(),
    'l ankle_Y': list(),
    'pelvis_X': list(),
    'pelvis_Y': list(),
    'thorax_X': list(),
    'thorax_Y': list(),
    'upper neck_X': list(),
    'upper neck_Y': list(),
    'head top_X': list(),
    'head top_Y': list(),
    'r wrist_X': list(),
    'r wrist_Y': list(),
    'r elbow_X': list(),
    'r elbow_Y': list(),
    'r shoulder_X': list(),
    'r shoulder_Y': list(),
    'l shoulder_X': list(),
    'l shoulder_Y': list(),
    'l elbow_X': list(),
    'l elbow_Y': list(),
    'l wrist_X': list(),
    'l wrist_Y': list(),
    'head box_x1': list(),
    'head box_y1': list(),
    'head box_x2': list(),
    'head box_y2': list(),
    'Category': list()
}

torsopoints = (2, 3, 12, 13)

for i in tqdm(range(annolist.shape[1])):
  if img_train[0, i]: #Just want the training images
    items = annolist[0, i]
    annorect = items.__dict__['annorect']
    if annorect.shape[0]:
      img = items.__dict__['image']
      img = img[0,0]
      name = img.__dict__['name']
      name = name[0]
      cat = categories[i,0].__dict__['cat_name']
      for j in range(annorect.shape[1]):
        points = annorect[0,j]
        datapoints["NAME"].append(name)
        datapoints['head box_x1'].append(points.__dict__['x1'][0][0])
        datapoints['head box_y1'].append(points.__dict__['y1'][0][0])
        datapoints['head box_x2'].append(points.__dict__['x2'][0][0])
        datapoints['head box_y2'].append(points.__dict__['y2'][0][0])
        if cat.shape[0] != 0:
          datapoints['Category'].append(cat[0])
        else:
          datapoints['Category'].append('bad')
        if 'annopoints' in points._fieldnames:
          annopoints = points.__dict__['annopoints']
          if annopoints.shape[0] != 0:
            points = annopoints[0,0]
            points = points.__dict__['point']
            visibles = {}
            for k in range(points.shape[1]):
              if 'is_visible' in points[0,k]._fieldnames:
                is_visible = points[0,k].__dict__['is_visible']
                if is_visible.size != 0:
                  is_visible = is_visible[0]
                  if is_visible == 1 or is_visible == '1' or points[0, k].__dict__['id'][0,0] in torsopoints: # Consider shoulders and hips as always visible or the torso bounding boxes will be very bad
                    id = points[0,k].__dict__['id'][0,0]
                    x = points[0,k].__dict__['x'][0,0]
                    y = points[0,k].__dict__['y'][0,0]
                    visibles[id] = (x, y)

            if 0 in visibles.keys():
              datapoints['r ankle_X'].append(visibles[0][0])
              datapoints['r ankle_Y'].append(visibles[0][1])
            else:
              datapoints['r ankle_X'].append(-1)
              datapoints['r ankle_Y'].append(-1)

            if 1 in visibles.keys():
              datapoints['r knee_X'].append(visibles[1][0])
              datapoints['r knee_Y'].append(visibles[1][1])
            else: 
              datapoints['r knee_X'].append(-1)
              datapoints['r knee_Y'].append(-1)

            if 2 in visibles.keys():
              datapoints['r hip_X'].append(visibles[2][0])
              datapoints['r hip_Y'].append(visibles[2][1])
            else:
              datapoints['r hip_X'].append(-1)
              datapoints['r hip_Y'].append(-1)

            if 3 in visibles.keys():
              datapoints['l hip_X'].append(visibles[3][0])
              datapoints['l hip_Y'].append(visibles[3][1])
            else:
              datapoints['l hip_X'].append(-1)
              datapoints['l hip_Y'].append(-1)
            
            if 4 in visibles.keys():
              datapoints['l knee_X'].append(visibles[4][0])
              datapoints['l knee_Y'].append(visibles[4][1])
            else:
              datapoints['l knee_X'].append(-1)
              datapoints['l knee_Y'].append(-1)
            
            if 5 in visibles.keys():
              datapoints['l ankle_X'].append(visibles[5][0])
              datapoints['l ankle_Y'].append(visibles[5][1])
            else:
              datapoints['l ankle_X'].append(-1)
              datapoints['l ankle_Y'].append(-1)

            if 6 in visibles.keys():
              datapoints['pelvis_X'].append(visibles[6][0])
              datapoints['pelvis_Y'].append(visibles[6][1])
            else:
              datapoints['pelvis_X'].append(-1)
              datapoints['pelvis_Y'].append(-1)

            if 7 in visibles.keys():
              datapoints['thorax_X'].append(visibles[7][0])
              datapoints['thorax_Y'].append(visibles[7][1])
            else:
              datapoints['thorax_X'].append(-1)
              datapoints['thorax_Y'].append(-1)
            
            if 8 in visibles.keys():
              datapoints['upper neck_X'].append(visibles[8][0])
              datapoints['upper neck_Y'].append(visibles[8][1])
            else:
              datapoints['upper neck_X'].append(-1)
              datapoints['upper neck_Y'].append(-1)
            
            if 9 in visibles.keys():
              datapoints['head top_X'].append(visibles[9][0])
              datapoints['head top_Y'].append(visibles[9][1])
            else:
              datapoints['head top_X'].append(-1)
              datapoints['head top_Y'].append(-1)
            
            if 10 in visibles.keys():
              datapoints['r wrist_X'].append(visibles[10][0])
              datapoints['r wrist_Y'].append(visibles[10][1])
            else:
              datapoints['r wrist_X'].append(-1)
              datapoints['r wrist_Y'].append(-1)

            if 11 in visibles.keys():
              datapoints['r elbow_X'].append(visibles[11][0])
              datapoints['r elbow_Y'].append(visibles[11][1])
            else:
              datapoints['r elbow_X'].append(-1)
              datapoints['r elbow_Y'].append(-1)
            
            if 12 in visibles.keys():
              datapoints['r shoulder_X'].append(visibles[12][0])
              datapoints['r shoulder_Y'].append(visibles[12][1])
            else:
              datapoints['r shoulder_X'].append(-1)
              datapoints['r shoulder_Y'].append(-1)
            
            if 13 in visibles.keys():
              datapoints['l shoulder_X'].append(visibles[13][0])
              datapoints['l shoulder_Y'].append(visibles[13][1])
            else:
              datapoints['l shoulder_X'].append(-1)
              datapoints['l shoulder_Y'].append(-1)
            
            if 14 in visibles.keys():
              datapoints['l elbow_X'].append(visibles[14][0])
              datapoints['l elbow_Y'].append(visibles[14][1])
            else:
              datapoints['l elbow_X'].append(-1)
              datapoints['l elbow_Y'].append(-1)

            if 15 in visibles.keys():
              datapoints['l wrist_X'].append(visibles[15][0])
              datapoints['l wrist_Y'].append(visibles[15][1])
            else:
              datapoints['l wrist_X'].append(-1)
              datapoints['l wrist_Y'].append(-1)
          else:
            for joint in joints:
              datapoints[joint].append(-1)
        else:
          for joint in joints:
            datapoints[joint].append(-1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:76: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Check that all lengths are equal

In [150]:
for key in datapoints.keys():
  print(len(datapoints[key]))

29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116
29116


In [151]:
dataset = pd.DataFrame(datapoints)
dataset

,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,l knee_X,l knee_Y,l ankle_X,l ankle_Y,pelvis_X,pelvis_Y,thorax_X,thorax_Y,upper neck_X,upper neck_Y,head top_X,head top_Y,r wrist_X,r wrist_Y,r elbow_X,r elbow_Y,r shoulder_X,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,head box_x1,head box_y1,head box_x2,head box_y2,Category
0,015601864.jpg,620,394,616,269,573,185,647,188,661,221,656,231,-1,-1,647,176,-1,-1,-1,-1,606,217,553,161,601,167,692,185,693,240,688,313,627,100,706,198,sports
1,015601864.jpg,895,293,910,279,945,223,1012,218,961,315,960,403,-1,-1,-1,-1,-1,-1,-1,-1,871,304,883,229,888,174,924,206,1013,203,955,263,841,145,902,228,sports
2,015599452.jpg,-1,-1,-1,-1,806,543,720,593,-1,-1,-1,-1,763,568,-1,-1,-1,-1,-1,-1,563,296,555,410,647,281,719,299,711,516,545,466,607,70,752,255,sports
3,015599452.jpg,-1,-1,-1,-1,987,607,1194,571,-1,-1,-1,-1,1091,589,1038,292,-1,-1,-1,-1,-1,-1,955,470,931,315,1145,269,1226,475,1096,433,903,73,1070,263,sports
4,015599452.jpg,-1,-1,-1,-1,228,537,74,536,-1,-1,-1,-1,151,537,129,251,-1,-1,-1,-1,-1,-1,297,456,232,251,26,251,26,423,-1,-1,27,36,186,214,sports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29111,084761779.jpg,362,350,347,297,350,224,317,226,-1,-1,336,312,334,225,312,173,-1,-1,-1,-1,-1,-1,-1,-1,335,168,288,177,-1,-1,-1,-1,287,130,324,163,transportation
29112,084761779.jpg,457,324,420,282,403,219,379,222,-1,-1,379,314,391,221,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,380,172,349,171,-1,-1,-1,-1,333,137,361,162,transportation
29113,084761779.jpg,-1,-1,-1,-1,468,221,449,226,461,271,500,315,459,224,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,432,175,403,179,-1,-1,-1,-1,400,140,428,168,transportation
29114,084761779.jpg,-1,-1,280,287,277,224,248,225,222,280,-1,-1,263,225,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,251,178,214,179,190,202,-1,-1,212,153,244,173,transportation


This dataset contains a lot of categories that don't involve much movement, and have obscured limbs. I'm removing ones like "inactivity quiet/light" as I don't expect them to be as useful. *Note: variables and files are named something along the lines of sports as the original idea was to just use the sports category. 

In [152]:
Sports = set()
for i in tqdm(dataset.index):
  if dataset["Category"][i] == "fishing and hunting" or dataset["Category"][i] == "home activities" or dataset["Category"][i] == "home repair" or dataset["Category"][i] == "inactivity quiet/light" or dataset["Category"][i] == "lawn and garden" or dataset["Category"][i] == "miscellaneous" or dataset["Category"][i] == "music playing" or dataset["Category"][i] == "occupation" or dataset["Category"][i] == "religious activities" or dataset["Category"][i] == "self care" or dataset["Category"][i] == "transportation" or dataset["Category"][i] == "volunteer activities":
    dataset.drop(i, inplace=True)
  else:
    Sports.add(dataset["NAME"][i])

In [153]:
dataset

,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,l knee_X,l knee_Y,l ankle_X,l ankle_Y,pelvis_X,pelvis_Y,thorax_X,thorax_Y,upper neck_X,upper neck_Y,head top_X,head top_Y,r wrist_X,r wrist_Y,r elbow_X,r elbow_Y,r shoulder_X,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,head box_x1,head box_y1,head box_x2,head box_y2,Category
0,015601864.jpg,620,394,616,269,573,185,647,188,661,221,656,231,-1,-1,647,176,-1,-1,-1,-1,606,217,553,161,601,167,692,185,693,240,688,313,627,100,706,198,sports
1,015601864.jpg,895,293,910,279,945,223,1012,218,961,315,960,403,-1,-1,-1,-1,-1,-1,-1,-1,871,304,883,229,888,174,924,206,1013,203,955,263,841,145,902,228,sports
2,015599452.jpg,-1,-1,-1,-1,806,543,720,593,-1,-1,-1,-1,763,568,-1,-1,-1,-1,-1,-1,563,296,555,410,647,281,719,299,711,516,545,466,607,70,752,255,sports
3,015599452.jpg,-1,-1,-1,-1,987,607,1194,571,-1,-1,-1,-1,1091,589,1038,292,-1,-1,-1,-1,-1,-1,955,470,931,315,1145,269,1226,475,1096,433,903,73,1070,263,sports
4,015599452.jpg,-1,-1,-1,-1,228,537,74,536,-1,-1,-1,-1,151,537,129,251,-1,-1,-1,-1,-1,-1,297,456,232,251,26,251,26,423,-1,-1,27,36,186,214,sports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29049,076912890.jpg,547,559,592,460,502,424,472,416,-1,-1,-1,-1,487,420,-1,-1,-1,-1,-1,-1,616,294,611,357,550,307,505,293,-1,-1,-1,-1,530,214,593,286,sports
29050,076912890.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,939,542,835,587,787,687,761,628,844,418,939,517,sports
29051,099616030.jpg,-1,-1,-1,-1,744,281,742,301,685,323,696,363,-1,-1,-1,-1,-1,-1,-1,-1,716,238,759,252,776,240,773,259,766,292,739,281,757,202,794,235,sports
29052,025398745.jpg,-1,-1,-1,-1,748,195,772,171,748,221,793,297,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,650,204,670,163,691,128,693,152,655,184,632,110,676,158,sports


In [135]:
len(dataset)

16614

16614 is the number of people with at least one body part visible, which should be enough for now.
Save this to csv for later processing.

In [154]:
!rm sports_annotations_no_box.csv

In [155]:
dataset.to_csv("sports_annotations_no_box.csv")

In [137]:
!rm -rf images_data

In [138]:
!mkdir images_data

In [141]:
for image in tqdm(os.listdir('images')):
  if image in Sports:
    shutil.copy(f'images/{image}', 'images_data')

In [142]:
len(os.listdir('images_data'))

9052

Lastly, I download and save this data to Google Drive for easier access in other files/programs. These files will also be found in the respository, with the exceptions of the tar file containing the images. Github doesn't like large files so you can access the images from this link https://drive.google.com/file/d/1-0yYrZAs-BoonZeXHhS5Cb5jWHzlNFvK/view?usp=sharing.

In [144]:
from google.colab import drive

In [145]:
drive.mount('/content/drive')
with open(f'/content/drive/My Drive/CPEN 291 Datasets/Sports_Dataset/sports_annotations_no_box.csv', 'wb'):
  shutil.copy("sports_annotations_no_box.csv", "/content/drive/My Drive/CPEN 291 Datasets/Sports_Dataset/sports_annotations_no_box.csv")
drive.flush_and_unmount()

Mounted at /content/drive


In [146]:
!tar cfz images.tar.gz images_data/
drive.mount('/content/drive')

with open(f'/content/drive/My Drive/CPEN 291 Datasets/Sports_Dataset/images.tar.gz', 'wb'):
  shutil.copy('images.tar.gz', '/content/drive/My Drive/CPEN 291 Datasets/Sports_Dataset/images.tar.gz')


drive.flush_and_unmount()

Mounted at /content/drive
